# 比赛

In [1]:
# coding: utf-8
from psutil import cpu_stats
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cbt
import category_encoders as ce
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split

In [2]:
os.listdir('../data/')

['submission.csv', 'test.csv', 'train.csv', 'train_label.csv']

In [3]:
submission = pd.read_csv('../data/submission.csv')
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
train_label = pd.read_csv('../data/train_label.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cpu_stats()

scpustats(ctx_switches=2984395770, interrupts=2264151210, soft_interrupts=0, syscalls=2791995793)

In [5]:
train.head()

,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,...,应收账款_年末数,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,00:00.0,5154.0,...,142.310005,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,00:00.0,7214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13:31.0,2110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,0.0,42:05.0,9.0,0.0,266000,NaN,19.160223,00:00.0,5163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,0.0,NaN,2.0,0.0,266400,NaN,19.210446,00:00.0,5439.0,...,156.901030,0.0,0.0,0.0,0.0,0.0,10.044194,0.508236,319.205293,0.0


In [6]:
np.where(train.isnull().sum()/train.shape[0]<0.5 )[0]

array([ 0,  1,  3,  4,  5,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21], dtype=int64)

In [7]:
columns = ['ID', '企业类型', '登记机关', '企业状态', '邮政编码', '注册资本', '核准日期', '行业代码', '经营期限自',
       '成立日期', '行业门类', '企业类别', '管辖机关', '经营范围', '增值税', '企业所得税', '印花税', '教育费',
       '城建税']

In [34]:
train[columns].isnull().sum()/train.shape[0]

ID       0.000000
企业类型     0.053460
登记机关     0.053460
企业状态     0.053460
邮政编码     0.349993
注册资本     0.054086
核准日期     0.053808
行业代码     0.053947
经营期限自    0.056105
成立日期     0.053599
行业门类     0.053599
企业类别     0.053460
管辖机关     0.053529
经营范围     0.000000
增值税      0.045385
企业所得税    0.200195
印花税      0.092301
教育费      0.000000
城建税      0.041139
dtype: float64

In [8]:
train['经营范围'].map(lambda x: len(x)).head()

0    244
1    126
2     53
3     59
4    248
Name: 经营范围, dtype: int64

In [9]:
feature = [ '企业类型', '登记机关', '企业状态', '注册资本', '行业代码', 
        '行业门类', '企业类别', '管辖机关', '经营范围', '增值税', '企业所得税', '印花税', '教育费',
       '城建税']

In [10]:
train[feature].head()

,企业类型,登记机关,企业状态,注册资本,行业代码,行业门类,企业类别,管辖机关,经营范围,增值税,企业所得税,印花税,教育费,城建税
0,9.0,1.0,0.0,19.110000,5154.0,1.0,0.0,1.0,"[1412, 1024, 21375, 15023, 18526, 18789, 17022...",106.039650,0.331552,1.247218,3.181189,7.422776
1,11.0,5.0,0.0,19.210446,7214.0,0.0,2.0,3.0,"[3671, 24856, 13525, 21079, 13219, 13694, 1442...",39.175600,51.244655,0.014363,1.175269,2.742293
2,0.0,3.0,0.0,19.120045,2110.0,2.0,0.0,4.0,"[3671, 14394, 23662, 11227, 19205, 15633, 5344...",0.000000,0.000000,0.541067,4.637491,10.060346
3,0.0,9.0,0.0,19.160223,5163.0,1.0,0.0,8.0,"[3287, 17425, 7503, 19922, 12217, 5089, 24884,...",0.027578,0.000000,0.008166,0.001318,0.001930
4,0.0,2.0,0.0,19.210446,5439.0,4.0,0.0,2.0,"[10573, 2876, 12295, 12295, 12295, 12295, 1229...",29.784126,1.134480,0.001004,0.893524,2.084889


In [11]:
train = train.merge(train_label,on='ID',how = 'left')

In [12]:
data = train.append(test)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [13]:
data['经营范围'] = data['经营范围'].map(lambda x : len(x))

In [14]:
object_col = ['企业类型','行业门类', '企业类别', '管辖机关']
for i in tqdm(object_col):  # 进度条
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))
    data[i] = data[i]
cpu_stats()

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 43.98it/s]


scpustats(ctx_switches=2984550045, interrupts=2264253933, soft_interrupts=0, syscalls=2793023244)

In [16]:
data[feature].head()

,企业类型,登记机关,企业状态,注册资本,行业代码,行业门类,企业类别,管辖机关,经营范围,增值税,企业所得税,印花税,教育费,城建税
0,83,1.0,0.0,19.110000,5154.0,1,0,1,244,106.039650,0.331552,1.247218,3.181189,7.422776
1,3,5.0,0.0,19.210446,7214.0,0,2,6,126,39.175600,51.244655,0.014363,1.175269,2.742293
2,0,3.0,0.0,19.120045,2110.0,12,0,7,53,0.000000,0.000000,0.541067,4.637491,10.060346
3,0,9.0,0.0,19.160223,5163.0,1,0,11,59,0.027578,0.000000,0.008166,0.001318,0.001930
4,0,2.0,0.0,19.210446,5439.0,14,0,5,248,29.784126,1.134480,0.001004,0.893524,2.084889


In [42]:
data['企业所得税与增值税之比'] = data['增值税']/data['企业所得税']

In [43]:
feature += ['企业所得税与增值税之比']

In [44]:
tr_index = ~data['Label'].isnull()
train = data[tr_index].reset_index(drop=True)
y = data[tr_index]['Label'].reset_index(drop=True).astype(int)
test = data[~tr_index].reset_index(drop=True)
print(train.shape,test.shape)

(14366, 180) (9578, 180)


In [45]:
from sklearn.metrics import roc_auc_score

In [46]:
def lgb_roc_auc_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) 
    return 'f1', roc_auc_score(y_true, y_hat), True

In [47]:
lgb_paras = {'objective': 'binary',
             'learning_rate': 0.1 ,
             'max_depth': 6 ,
             'feature_fraction': 0.8, 
             'bagging_fraction' : 0.8,
             'num_threads':-1}

In [48]:
fi = []
cv_score = []
test_pred = np.zeros((test.shape[0],))
skf = StratifiedKFold(n_splits=5, random_state=2019, shuffle=True)

In [49]:
for index, (train_index, test_index) in enumerate(skf.split(train, y)):
    print(index)
    train_x, test_x, train_y, test_y = train.iloc[train_index],train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    lgb_model = lgb.train(lgb_paras,
                          train_set = lgb.Dataset(train_x[feature], train_y),
                          valid_sets=[lgb.Dataset(test_x[feature],test_y)],
                          
                          num_boost_round=800 ,
                          feval=lgb_roc_auc_score,
                          verbose_eval=50,
                          categorical_feature = object_col
                          )

    

    y_val = lgb_model.predict(test_x[feature])
    
    print( roc_auc_score( test_y , np.round( y_val) ) )
    
    
    cv_score.append(roc_auc_score(test_y,np.round(y_val)))
    
    print(cv_score[index])
    
    test_pred += lgb_model.predict(test[feature]) / 5
        
cpu_stats()


0


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1161: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['企业类别', '企业类型', '管辖机关', '行业门类']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[50]	valid_0's binary_logloss: 0.162648	valid_0's f1: 0.514794
[100]	valid_0's binary_logloss: 0.170058	valid_0's f1: 0.522024
[150]	valid_0's binary_logloss: 0.17755	valid_0's f1: 0.544158
[200]	valid_0's binary_logloss: 0.184095	valid_0's f1: 0.559472
[250]	valid_0's binary_logloss: 0.189648	valid_0's f1: 0.569309
[300]	valid_0's binary_logloss: 0.198754	valid_0's f1: 0.571024
[350]	valid_0's binary_logloss: 0.207402	valid_0's f1: 0.576501
[400]	valid_0's binary_logloss: 0.215788	valid_0's f1: 0.583134
[450]	valid_0's binary_logloss: 0.224053	valid_0's f1: 0.58332
[500]	valid_0's binary_logloss: 0.231047	valid_0's f1: 0.575384
[550]	valid_0's binary_logloss: 0.240345	valid_0's f1: 0.572552
[600]	valid_0's binary_logloss: 0.248321	valid_0's f1: 0.576912
[650]	valid_0's binary_logloss: 0.255999	valid_0's f1: 0.579743
[700]	valid_0's binary_logloss: 0.263945	valid_0's f1: 0.579743
[750]	valid_0's binary_logloss: 0.272888	valid_0's f1: 0.579371
[800]	valid_0's binary_logloss: 0.278379	va

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1161: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['企业类别', '企业类型', '管辖机关', '行业门类']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[50]	valid_0's binary_logloss: 0.146684	valid_0's f1: 0.520159
[100]	valid_0's binary_logloss: 0.151917	valid_0's f1: 0.540132
[150]	valid_0's binary_logloss: 0.157976	valid_0's f1: 0.551754
[200]	valid_0's binary_logloss: 0.164599	valid_0's f1: 0.556515
[250]	valid_0's binary_logloss: 0.171724	valid_0's f1: 0.571355
[300]	valid_0's binary_logloss: 0.178674	valid_0's f1: 0.574014
[350]	valid_0's binary_logloss: 0.18475	valid_0's f1: 0.589413
[400]	valid_0's binary_logloss: 0.192097	valid_0's f1: 0.586754
[450]	valid_0's binary_logloss: 0.198589	valid_0's f1: 0.591514
[500]	valid_0's binary_logloss: 0.206692	valid_0's f1: 0.604626
[550]	valid_0's binary_logloss: 0.215037	valid_0's f1: 0.596461
[600]	valid_0's binary_logloss: 0.220256	valid_0's f1: 0.604067
[650]	valid_0's binary_logloss: 0.226639	valid_0's f1: 0.596275
[700]	valid_0's binary_logloss: 0.233855	valid_0's f1: 0.590397
[750]	valid_0's binary_logloss: 0.242771	valid_0's f1: 0.603695
[800]	valid_0's binary_logloss: 0.250913	v

...

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1161: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['企业类别', '企业类型', '管辖机关', '行业门类']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[50]	valid_0's binary_logloss: 0.145728	valid_0's f1: 0.51867
[100]	valid_0's binary_logloss: 0.150539	valid_0's f1: 0.521009
[150]	valid_0's binary_logloss: 0.157284	valid_0's f1: 0.532445
[200]	valid_0's binary_logloss: 0.164514	valid_0's f1: 0.544998
[250]	valid_0's binary_logloss: 0.17293	valid_0's f1: 0.552045
[300]	valid_0's binary_logloss: 0.18032	valid_0's f1: 0.556806
[350]	valid_0's binary_logloss: 0.188096	valid_0's f1: 0.565768
[400]	valid_0's binary_logloss: 0.196618	valid_0's f1: 0.570715
[450]	valid_0's binary_logloss: 0.204687	valid_0's f1: 0.572816
[500]	valid_0's binary_logloss: 0.210945	valid_0's f1: 0.569598
[550]	valid_0's binary_logloss: 0.218455	valid_0's f1: 0.574917
[600]	valid_0's binary_logloss: 0.226844	valid_0's f1: 0.564278
[650]	valid_0's binary_logloss: 0.232189	valid_0's f1: 0.57739
[700]	valid_0's binary_logloss: 0.238178	valid_0's f1: 0.577018
[750]	valid_0's binary_logloss: 0.244379	valid_0's f1: 0.574544
[800]	valid_0's binary_logloss: 0.251523	vali

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1161: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['企业类别', '企业类型', '管辖机关', '行业门类']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:725: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[50]	valid_0's binary_logloss: 0.146882	valid_0's f1: 0.543031
[100]	valid_0's binary_logloss: 0.151188	valid_0's f1: 0.543642
[150]	valid_0's binary_logloss: 0.157367	valid_0's f1: 0.55279
[200]	valid_0's binary_logloss: 0.163919	valid_0's f1: 0.561753
[250]	valid_0's binary_logloss: 0.17168	valid_0's f1: 0.563481
[300]	valid_0's binary_logloss: 0.179083	valid_0's f1: 0.576034
[350]	valid_0's binary_logloss: 0.185483	valid_0's f1: 0.594517
[400]	valid_0's binary_logloss: 0.194152	valid_0's f1: 0.591485
[450]	valid_0's binary_logloss: 0.202529	valid_0's f1: 0.590927
[500]	valid_0's binary_logloss: 0.210943	valid_0's f1: 0.576831
[550]	valid_0's binary_logloss: 0.220544	valid_0's f1: 0.576459
[600]	valid_0's binary_logloss: 0.227816	valid_0's f1: 0.573986
[650]	valid_0's binary_logloss: 0.236338	valid_0's f1: 0.576831
[700]	valid_0's binary_logloss: 0.244765	valid_0's f1: 0.573241
[750]	valid_0's binary_logloss: 0.252581	valid_0's f1: 0.573427
[800]	valid_0's binary_logloss: 0.260603	va

scpustats(ctx_switches=1313470056, interrupts=918837846, soft_interrupts=0, syscalls=3371691635)

In [50]:
submission['Label'] = test_pred

submission.to_csv('submission.csv',index=False)